In [ ]:
import sys
import networkx as nx
import pickle
import itertools as it
import os
import shutil

import MDAnalysis as mda
import MDAnalysis.transformations as trans
import nglview as nv


import warnings
warnings.filterwarnings(action='once')


if '/home/anna/Documents/cinnabar' not in sys.path:
    sys.path.insert(1, '/home/anna/Documents/cinnabar')
import cinnabar

print(cinnabar.__file__)

print("adding code to the pythonpath...")
code = '/home/anna/Documents/code/python'
if code not in sys.path:
    sys.path.insert(1, code)
import pipeline

from pipeline import *
from pipeline.analysis import *
from pipeline.prep import *
from pipeline.utils import *


In [ ]:
prot_file = "/home/anna/Documents/benchmark/tyk2_benchmark/execution_model/protocol_2fs.dat"
protocol = pipeline_protocol(prot_file)
protocol.validate()
protocol.fepprep("both")

protocol.num_lambda(11)
protocol.engine("AMBER")

In [ ]:
# Load equilibrated inputs for both ligands
prep_dir = "/home/anna/Documents/benchmark/tyk2_benchmark/prep"
lig_1 = "lig_ejm31"
lig_2 = "lig_ejm48"

fepprep_obj = fepprep(protocol=protocol)

for name, leg in zip(["lig", "sys"], ["free", "bound"]):

    # Load equilibrated inputs for both ligands
    system_1 = BSS.IO.readMolecules(
        [f"{prep_dir}/{lig_1}_{name}_equil_solv.rst7", f"{prep_dir}/{lig_1}_{name}_equil_solv.prm7"])
    system_2 = BSS.IO.readMolecules(
        [f"{prep_dir}/{lig_2}_{name}_equil_solv.rst7", f"{prep_dir}/{lig_2}_{name}_equil_solv.prm7"])

    fepprep_obj.add_system(system_1, free_bound=leg, start_end="start")
    fepprep_obj.add_system(system_2, free_bound=leg, start_end="end")

folder = f"/home/anna/Documents/code/test/both_{lig_1}~{lig_2}"
fepprep_obj.generate_folders(f"{folder}")

In [ ]:
fepprep_obj = fepprep(system_free, system_bound, protocol)

In [ ]:
fepprep_obj._free_system_0

In [ ]:
sys3

In [ ]:
BSS.Notebook.View(fepprep_obj._free_system_0).system()

In [ ]:
pmemd_path = os.environ["AMBERHOME"] + "/bin/pmemd.cuda"

In [ ]:
fepprep_obj = fepprep(system_free, system_bound, protocol)
folder = f"/home/anna/Documents/code/test/{lig_1}~{lig_2}_restraints"
free_system, bound_system = fepprep_obj.prep_system_middle(pmemd_path, work_dir=f"{folder}/eq")

# fepprep_obj.generate_folders(f"{folder}")

In [ ]:

fepprep_obj.generate_folders(f"{folder}")

In [ ]:
mol_eq = free_system.getPerturbableMolecules()[0].copy()

In [ ]:
mol_non = system_free.getPerturbableMolecules()[0]

In [ ]:
both_mols = mol_eq + mol_non
both_mols

In [ ]:
BSS.Notebook.View(both_mols).molecules()

In [ ]:
system_free.nPerturbableMolecules()

In [ ]:
BSS.Notebook.View(system_bound).system()


In [ ]:
fepprep_obj2 = fepprep(system_free, system_bound, protocol)
folder2 = f"/home/anna/Documents/code/test/{lig_1}~{lig_2}_noeq"
fepprep_obj2.generate_folders(f"{folder2}")

In [ ]:
fepprep_obj.generate_folders(f"{folder}")

In [ ]:
lig_fep = "fepprep"
engine="AMBER"

protocol_min_rest, protocol_min = min_prots(lig_fep)

protocol_nvt_sol, protocol_nvt_heavy, protocol_nvt = nvt_prots(lig_fep)

protocol_npt_heavy, protocol_npt_heavy_lighter, protocol_npt = npt_prots(lig_fep)

In [ ]:
protocol_min.getLambdaValues()

In [ ]:
minimised1 = run_process(system_free, protocol_min_rest, engine, pmemd_path)
minimised2 = run_process(minimised1, protocol_min, engine, pmemd_path, work_dir=folder)

In [ ]:
BSS.IO.saveMolecules(f"{folder}/minimised", minimised2, "PDB")

In [ ]:
BSS.IO.savePerturbableSystem(f"{folder}/minimised", minimised2)

In [ ]:
equil1 = run_process(equil3, protocol_nvt_sol, engine, pmemd_path) #, work_dir=f"{folder}/equil1"


In [ ]:
equil2 = run_process(equil1, protocol_nvt_heavy, engine, pmemd_path) #, work_dir=f"{folder}/equil2"


In [ ]:
equil3a = run_process(equil2, protocol_nvt, engine, pmemd_path, work_dir=f"{folder}/equil3")

In [ ]:
equil3 = run_process(minimised2, protocol_nvt, engine, pmemd_path, work_dir=f"{folder}/min_to_equil3")

In [ ]:
BSS.Notebook.View(equil4).system()

In [ ]:
equil4 = run_process(equil3, heat_protocol, engine, pmemd_path, work_dir=f"{folder}/eq_test")

In [ ]:
BSS.Notebook.View(merge.extract_ligand(equil4)).system()

In [ ]:
protocol_npt_heavy

In [ ]:
equil4 = run_process(equil3, protocol_npt_heavy, engine, pmemd_path, work_dir=f"{folder}/eq_test2")


In [ ]:
equil5 = run_process(equil4, protocol_npt_heavy_lighter, engine, pmemd_path, work_dir=f"{folder}/equil5")

In [ ]:
sys_equil_fin = run_process(equil3, protocol_npt, engine, pmemd_path, work_dir=f"{folder}/equilfinal")

In [ ]:
fepprep_obj = fepprep(system_free, system_bound, protocol)
free_system, bound_system = fepprep_obj.prep_system_middle(pmemd_path)
# fepprep_obj.generate_folders("/home/anna/Documents/code/test/lig_ejm31~lig_ejm48")

In [ ]:
args = {"lam":0.5, "lam_vals":[0.5]}

min_protocol = BSS.Protocol.FreeEnergyMinimisation(
                                                num_lam=protocol.num_lambda(),
                                                steps=protocol.min_steps(), **args
                                                )
heat_protocol = BSS.Protocol.FreeEnergyEquilibration(timestep=protocol.timestep()*protocol.timestep_unit(),
                                                    num_lam=protocol.num_lambda(),
                                                    runtime=protocol.eq_runtime()*protocol.eq_runtime_unit(),
                                                    pressure=None,
                                                    temperature_start=protocol.start_temperature()*protocol.temperature_unit(),
                                                    temperature_end=protocol.end_temperature()*protocol.temperature_unit(),
                                                    hmr_factor=protocol.hmr_factor(), **args
                                                    )
eq_protocol = BSS.Protocol.FreeEnergyEquilibration(timestep=protocol.timestep()*protocol.timestep_unit(),
                                                num_lam=protocol.num_lambda(),
                                                runtime=protocol.eq_runtime()*protocol.eq_runtime_unit(),
                                                temperature=protocol.temperature()*protocol.temperature_unit(),
                                                pressure=protocol.pressure()*protocol.pressure_unit(),
                                                restart=True,
                                                hmr_factor=protocol.hmr_factor(), **args
                                                )
freenrg_protocol = BSS.Protocol.FreeEnergy(timestep=protocol.timestep()*protocol.timestep_unit(),
                                            num_lam=protocol.num_lambda(),
                                            runtime=protocol.sampling()*protocol.sampling_unit(),
                                            temperature=protocol.temperature()*protocol.temperature_unit(),
                                            pressure=protocol.pressure()*protocol.pressure_unit(),
                                            restart=True,
                                            hmr_factor=protocol.hmr_factor(), **args
                                        )

In [ ]:
minimised1 = run_process(system_free, min_protocol, "AMBER", pmemd_path)

In [ ]:
eq_sys = run_process(minimised1, heat_protocol, "AMBER", pmemd_path)

In [ ]:
eq2_sys = run_process(eq_sys, eq_protocol, "AMBER", pmemd_path)

In [ ]:
BSS.Notebook.View(eq_sys).system()

In [ ]:
final_sys = run_process(eq2_sys, freenrg_protocol, "AMBER", pmemd_path)

In [ ]:
free_system.nPerturbableMolecules()

In [ ]:
transf = ["lig_ejm31~lig_ejm45", "lig_ejm44~lig_ejm45","lig_ejm44~lig_ejm49"]
engine = "GROMACS"
main_dir = "/home/anna/Documents/benchmark/tyk2_benchmark"
# methods = ["1fs", "2fs_HMR4", "4fs_HMR4", "4fs_HMR3", "2fs_HMR3", "2fs"]
methods = ["2fs"]

# options
ana_file = f"{main_dir}/execution_model/analysis_protocol.dat"
analysis_options = analysis_protocol(ana_file, auto_validate=True)
# analysis_options.rewrite_protocol()

for method in methods:
    for trans in transf:
        path_to_dir = f"{main_dir}/outputs_extracted/{engine}/{method}/{trans}"
        print(path_to_dir)
        final_results_folder = f"{main_dir}/outputs_extracted/results"

        # try:
        # using the pipeline module for analysis
        analysed_pert = analyse(path_to_dir)
        analysed_pert.set_options(analysis_options)
        analysed_pert.set_options({"try pickle":False})
        avg, error, repeats_tuple_list = analysed_pert.analyse_all_repeats()
        # analysed_pert.plot_graphs()
        write_analysis_file(analysed_pert, final_results_folder, method=method)
        # except Exception as e:
        #     print(e)
        #     print(f"could not analyse {path_to_dir}")

In [ ]:
transf = ["lig_ejm31~lig_ejm45", "lig_ejm31~lig_ejm48","lig_ejm43~lig_ejm46", "lig_ejm46~lig_jmc28", "lig_jmc27~lig_jmc28"]
engine = "AMBER"
main_dir = "/home/anna/Documents/benchmark/tyk2_benchmark"
# methods = ["1fs", "2fs_HMR4", "4fs", "4fs_HMR3", "2fs_HMR3", "2fs"]
methods = ["both_failed_wrong_lambda_windows_used"]

# options
ana_file = f"{main_dir}/execution_model/analysis_protocol.dat"
analysis_options = analysis_protocol(ana_file, auto_validate=True)
# analysis_options.rewrite_protocol()

for method in methods:
    for trans in transf:
        # trans = f"{trans.split('~')[1]}~{trans.split('~')[0]}" # for reverse
        path_to_dir = f"{main_dir}/outputs_extracted/{engine}/{method}/{trans}"
        print(path_to_dir)
        final_results_folder = f"{main_dir}/outputs_extracted/results"

        try:
            # using the pipeline module for analysis
            analysed_pert = analyse(path_to_dir)
            analysed_pert.set_options(analysis_options)
            avg, error, repeats_tuple_list = analysed_pert.analyse_all_repeats()
            # analysed_pert.plot_graphs()
            write_analysis_file(analysed_pert, final_results_folder, method=method)
        except Exception as e:
            print(e)
            print(f"could not analyse {path_to_dir}")